<a href="https://colab.research.google.com/github/Ellisetr/Intelligent-Systems-and-Technologies/blob/main/%D0%98%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D0%B8_%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B8_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная 1-2
1. Применить для тестовых вариантов и вариантов из репозиториев различные алгоритмы поиска ассоциативных правил при одинаковых начальных
условиях (при одинаковых пороговых значениях для поддержки и достоверности) и сравнить полученные результаты. Для тестовых вариантов выполнить
ручные расчеты (например, с применением MS Excel) и расчеты с применением
программных библиотек на языке Python. Для вариантов из репозиториев выполнить расчеты с применением программных библиотек на языке Python.
В качестве алгоритмов поиска ассоциативных правил использовать алгоритмы:

*   Apriori;
*   Efficient Apriori;
*   FPGrowth.

2. Сформировать базы ассоциативных правил с уровнем минимальной достоверности 60% и 80%. Вычислить для ассоциативных правил поддержку, достоверность, значимость.
3. Оценить время формирования искомых ассоциативных правил с применением различных алгоритмов и построить диаграммы, позволяющие выполнить сравнительный анализ.
4. Выполнить визуализацию ассоциативных правил.

Установка либы в облако:

In [ ]:
%pip install --index-url https://test.pypi.org/simple/ PyARMViz
%pip install apriori_python
%pip install efficient-apriori
%pip install fpgrowth_py

Looking in indexes: https://test.pypi.org/simple/, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Открытие и получение dataset:

In [ ]:
import pandas as pd

class dataset:
    def __init__(self, url, cols):
        if url.split('.')[-1] == 'csv':
            self.df = pd.read_csv(url, usecols = cols)
        else:
            self.df = pd.read_excel(url, usecols = cols)
        self.df[cols[1]] = self.df[cols[1]].astype(str)

    def agregate_transactions(self):
        self.agregated_transactions = {}
        for i in range(self.df.shape[0]):
            if list(self.df.iloc[i])[0] not in self.agregated_transactions.keys():
                self.agregated_transactions[list(self.df.iloc[i])[0]] = [list(self.df.iloc[i])[1]]
            else:
                self.agregated_transactions[list(self.df.iloc[i])[0]].append(list(self.df.iloc[i])[1])
        self.agregated_transactions = list(self.agregated_transactions.values())

Apriori:

In [ ]:
from apriori_python import apriori

class Apriori:
    def __init__(self, dataset):
        self.transactions = dataset.agregated_transactions

    def create_rules(self, min_sup, min_conf):
        self.freq_item_set, self.rules = apriori(self.transactions, minSup=min_sup, minConf=min_conf)

    def return_rules(self):
        return self.freq_item_set, self.rules

    def print_rules(self):
        print("Apriori rules:",self.rules)
        print("Apriori freq:",self.freq_item_set)

Efficeint Apriori:

In [ ]:
from efficient_apriori import apriori as ef_apriori

class Efficient_apriori(Apriori):
    def __init__(self, dataset):
        super().__init__(dataset)

    def create_rules(self, min_sup, min_conf):
        self.freq_item_set, self.rules = ef_apriori(self.transactions, min_support=min_sup, min_confidence=min_conf)
    
    def return_rules(self):
        return self.freq_item_set, self.rules

    def print_rules(self):
        self.print_rules_help(2,1)
        print("Efficient apriori freq:",self.freq_item_set)

    def print_rules_help(self, lhs_len: int, rhs_len: int):
        rules_rhs = filter(lambda rule: len(rule.lhs) == lhs_len and len(rule.rhs) == rhs_len, self.rules)
        for rule in sorted(rules_rhs, key=lambda rule: rule.confidence):
            print(rule)

FPGrowth:

In [ ]:
from fpgrowth_py import fpgrowth

class Fpgrowth(Apriori):
    def __init__(self, dataset):
        super().__init__(dataset)

    def create_rules(self, min_sup, min_conf):
        self.freq_item_set, self.rules = fpgrowth(self.transactions, minSupRatio=min_sup, minConf=min_conf)

    def return_rules(self):
        return self.freq_item_set, self.rules
    
    def print_rules(self):
        print("FPGrowth rules:",self.rules)
        print("FPGrowth freq:",self.freq_item_set)

Конфигурация для данных из интернета

In [ ]:
from PyARMViz import PyARMViz
from PyARMViz import *

class lab:
  def __init__(self, url, cols, min_sup, min_conf):    
    self.min_sup = 0.02
    self.min_conf = 0.6
    # Открытие таблицы и инициализация
    self.Ds = dataset(url, cols)
    self.Ds.agregate_transactions()

  def fpg_out(self):
    self.fpg = Fpgrowth(self.Ds)
    self.fpg.create_rules(self.min_sup, self.min_conf)
    self.fpg.print_rules()

  def std_apr_out(self):
    self.std_apriori = Apriori(self.Ds)
    self.std_apriori.create_rules(self.min_sup, self.min_conf)
    self.std_apriori.print_rules()

  def eff_apr_out(self):
    self.eff_apriori = Efficient_apriori(self.Ds)
    self.eff_apriori.create_rules(self.min_sup, self.min_conf)
    self.eff_apriori.print_rules()

  def PyARM_out(self):
    PyARMViz.generate_rule_graph_plotly(self.eff_apriori.rules)

In [ ]:
internet = lab("https://raw.githubusercontent.com/viktree/curly-octo-chainsaw/master/BreadBasket_DMS.csv", ['Transaction', 'Item'],0.02, 0.6)

In [ ]:
internet2 = lab("https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx",['InvoiceNo', 'Description'], 0.02, 0.6)

FPGrowth

In [ ]:
internet.fpg_out()

FPGrowth rules: [[{'Toast'}, {'Coffee'}, 0.7044025157232704]]
FPGrowth freq: [{'Truffles'}, {'Scandinavian'}, {'Toast'}, {'Toast', 'Coffee'}, {'Scone'}, {'Coffee', 'Scone'}, {'Soup'}, {'Coffee', 'Soup'}, {'Juice'}, {'Coffee', 'Juice'}, {'Alfajores'}, {'Alfajores', 'Coffee'}, {'Muffin'}, {'Coffee', 'Muffin'}, {'Farm House'}, {'Brownie'}, {'Coffee', 'Brownie'}, {'Cookies'}, {'Coffee', 'Cookies'}, {'Hot chocolate'}, {'Coffee', 'Hot chocolate'}, {'Medialuna'}, {'Coffee', 'Medialuna'}, {'Sandwich'}, {'Sandwich', 'Bread'}, {'Sandwich', 'Coffee'}, {'NONE'}, {'NONE', 'Bread'}, {'NONE', 'Coffee'}, {'Pastry'}, {'Pastry', 'Bread'}, {'Pastry', 'Coffee'}, {'Cake'}, {'Bread', 'Cake'}, {'Tea', 'Cake'}, {'Coffee', 'Cake'}, {'Coffee', 'Cake'}, {'Tea'}, {'Bread', 'Tea'}, {'Coffee', 'Tea'}, {'Bread'}, {'Bread'}, {'Coffee', 'Bread'}, {'Coffee'}, {'Coffee'}]


In [ ]:
internet2.fpg_out()

FPGrowth rules: [[{'RED HANGING HEART T-LIGHT HOLDER'}, {'WHITE HANGING HEART T-LIGHT HOLDER'}, 0.6631299734748011], [{'CHARLOTTE BAG PINK POLKADOT'}, {'RED RETROSPOT CHARLOTTE BAG'}, 0.6921052631578948], [{'GARDENERS KNEELING PAD CUP OF TEA '}, {'GARDENERS KNEELING PAD KEEP CALM '}, 0.7176470588235294], [{'PINK REGENCY TEACUP AND SAUCER'}, {'ROSES REGENCY TEACUP AND SAUCER '}, 0.766541822721598], [{'PINK REGENCY TEACUP AND SAUCER'}, {'ROSES REGENCY TEACUP AND SAUCER ', 'GREEN REGENCY TEACUP AND SAUCER'}, 0.6853932584269663], [{'PINK REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER '}, {'GREEN REGENCY TEACUP AND SAUCER'}, 0.8941368078175895], [{'PINK REGENCY TEACUP AND SAUCER', 'GREEN REGENCY TEACUP AND SAUCER'}, {'ROSES REGENCY TEACUP AND SAUCER '}, 0.8524844720496895], [{'ROSES REGENCY TEACUP AND SAUCER ', 'GREEN REGENCY TEACUP AND SAUCER'}, {'PINK REGENCY TEACUP AND SAUCER'}, 0.7002551020408163], [{'PINK REGENCY TEACUP AND SAUCER'}, {'ROSES REGENCY TEACUP AND SAUCER ', '

Apriori

In [ ]:
internet.std_apr_out()

Apriori rules: [[{'Toast'}, {'Coffee'}, 0.7044025157232704]]
Apriori freq: {1: {frozenset({'Pastry'}), frozenset({'Bread'}), frozenset({'Sandwich'}), frozenset({'Farm House'}), frozenset({'Scone'}), frozenset({'Brownie'}), frozenset({'Cookies'}), frozenset({'Hot chocolate'}), frozenset({'Juice'}), frozenset({'Soup'}), frozenset({'Alfajores'}), frozenset({'Muffin'}), frozenset({'Toast'}), frozenset({'Tea'}), frozenset({'Coffee'}), frozenset({'Medialuna'}), frozenset({'NONE'}), frozenset({'Cake'}), frozenset({'Truffles'}), frozenset({'Scandinavian'})}, 2: {frozenset({'Coffee', 'Bread'}), frozenset({'Bread', 'Tea'}), frozenset({'Tea', 'Cake'}), frozenset({'Coffee', 'Tea'}), frozenset({'Coffee', 'Cake'}), frozenset({'Pastry', 'Coffee'}), frozenset({'Coffee', 'Medialuna'}), frozenset({'Coffee', 'NONE'}), frozenset({'Toast', 'Coffee'}), frozenset({'Pastry', 'Bread'}), frozenset({'Bread', 'Cake'}), frozenset({'NONE', 'Bread'}), frozenset({'Coffee', 'Juice'}), frozenset({'Coffee', 'Cookies'}),

In [ ]:
internet2.std_apr_out()

KeyboardInterrupt: ignored

Efficient Apriori

In [ ]:
internet.eff_apr_out()
internet.PyARM_out()

Efficient apriori freq: {1: {('Bread',): 3097, ('Scandinavian',): 275, ('Hot chocolate',): 552, ('Cookies',): 515, ('Muffin',): 364, ('Coffee',): 4528, ('Pastry',): 815, ('Medialuna',): 585, ('Tea',): 1350, ('NONE',): 753, ('Farm House',): 371, ('Juice',): 365, ('Soup',): 326, ('Cake',): 983, ('Sandwich',): 680, ('Alfajores',): 344, ('Brownie',): 379, ('Truffles',): 192, ('Toast',): 318, ('Scone',): 327}, 2: {('Bread', 'Cake'): 221, ('Bread', 'Coffee'): 852, ('Bread', 'NONE'): 196, ('Bread', 'Pastry'): 276, ('Bread', 'Tea'): 266, ('Cake', 'Coffee'): 518, ('Cake', 'Tea'): 225, ('Coffee', 'Cookies'): 267, ('Coffee', 'Hot chocolate'): 280, ('Coffee', 'Juice'): 195, ('Coffee', 'Medialuna'): 333, ('Coffee', 'NONE'): 401, ('Coffee', 'Pastry'): 450, ('Coffee', 'Sandwich'): 362, ('Coffee', 'Tea'): 472, ('Coffee', 'Toast'): 224}}


In [ ]:
internet2.eff_apr_out()
internet2.PyARM_out()

{GREEN REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER } -> {PINK REGENCY TEACUP AND SAUCER} (conf: 0.700, supp: 0.021, lift: 22.642, conv: 3.233)
{GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY TEACUP AND SAUCER} -> {ROSES REGENCY TEACUP AND SAUCER } (conf: 0.852, supp: 0.021, lift: 19.714, conv: 6.486)
{PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER } -> {GREEN REGENCY TEACUP AND SAUCER} (conf: 0.894, supp: 0.021, lift: 21.909, conv: 9.061)
Efficient apriori freq: {1: {('WHITE HANGING HEART T-LIGHT HOLDER',): 2302, ('ASSORTED COLOUR BIRD ORNAMENT',): 1467, ('HOME BUILDING BLOCK WORD',): 787, ('LOVE BUILDING BLOCK WORD',): 631, ('DOORMAT NEW ENGLAND',): 602, ('JAM MAKING SET WITH JARS',): 1220, ('ALARM CLOCK BAKELIKE PINK',): 799, ('ALARM CLOCK BAKELIKE RED ',): 1081, ('ALARM CLOCK BAKELIKE GREEN',): 1001, ('VINTAGE HEADS AND TAILS CARD GAME ',): 611, ('ROUND SNACK BOXES SET OF4 WOODLAND ',): 749, ('SPACEBOY LUNCH BOX ',): 908, ('LUNCH BOX I LOVE LONDON',): 